In [23]:
from IPython.display import display, Markdown

import pandas as pd
import gc
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import make_pipeline

from sklearn.model_selection import RandomizedSearchCV, train_test_split, KFold, cross_val_score

from sklearn.metrics import roc_auc_score

from hyperopt import hp, fmin, tpe, Trials, STATUS_OK, space_eval

from sklearn.tree import plot_tree

from sklearn.preprocessing import LabelEncoder, PolynomialFeatures, StandardScaler, OneHotEncoder

from scipy.sparse import hstack

# XGBoost
import xgboost as xgb

import numpy as np

import matplotlib.pyplot as plt

import itertools
from scipy.stats import uniform
import pandas as pd # Para cargar los datos y hacer OHE.
import numpy as np  # Para lidiar con NaNs.
import time
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import ParameterSampler
from sklearn.metrics import confusion_matrix

random_state = 42
np.random.seed(random_state)


In [24]:
# Load the competition data
comp_data = pd.read_csv("../competition_data.csv")

#aher OHE con pdp
comp_data['is_pdp'].fillna(-1, inplace=True)


In [25]:

comp_data = pd.get_dummies(comp_data, columns=['is_pdp'], prefix='is_pdp')

comp_data = pd.get_dummies(comp_data, columns=['listing_type_id','logistic_type','platform'])

In [26]:
comp_data = comp_data.drop(
    columns=['accepts_mercadopago',
            # 'available_quantity',
            # 'avg_gmv_item_domain_30days',
            # 'avg_gmv_item_sel',
            # 'avg_gmv_seller_bday',
            # 'avg_qty_orders_item_domain_30days',
            # 'avg_qty_orders_item_sel_30days',
            # 'avg_si_item_sel_30day',
            'benefit',
            'boosted',
            'category_id',
            # 'conversion',
            'date',             #chiche d efechas separar dia mes etc
            'deal_print_id',
            'domain_id',
            'etl_version',
            'free_shipping',    
            'fulfillment',
            'full_name',
            # 'health',
            # 'is_pdp',
            'product_id',
            'item_id',
            # 'listing_type_id', #hacer OHE
            # 'logistic_type',   #hacer OHE
            'main_picture',
            # 'offset',
            # 'original_price',
            # 'platform',        #hacer OHE
            # 'price',
            # 'print_position',
            'print_server_timestamp',
            # 'qty_items_dom',
            # 'qty_items_sel',
            'site_id',
            # 'sold_quantity',
            'tags',             #experimentar
            'title',            #W2vec
            # 'total_asp_item_domain_30days',
            # 'total_asp_item_sel_30days',
            # 'total_gmv_domain_bday',
            # 'total_gmv_item_30days',
            # 'total_items_domain',
            # 'total_items_seller',
            # 'total_orders_domain_30days',
            # 'total_orders_item_30days',
            # 'total_orders_sel_30days',
            # 'total_si_domain_30days',
            # 'total_si_item_30days',
            # 'total_si_sel_30days',
            # 'total_visits_domain',
            # 'total_visits_item',
            # 'total_visits_seller',
            'uid',
            'user_id',
            'warranty',
            # 'ROW_ID'
            ]
)

In [27]:

# Split into training and evaluation samples
train_data = comp_data[comp_data["ROW_ID"].isna()]
eval_data = comp_data[comp_data["ROW_ID"].notna()]
del comp_data
gc.collect()

y_train_all = train_data["conversion"]
X_train_all = train_data.drop(columns=["conversion", "ROW_ID"])
# X_train_all = X_train_all.select_dtypes(include='number') 
del train_data
gc.collect()

#hold out set
X_train, X_val, y_train, y_val = train_test_split(X_train_all, y_train_all, test_size=0.2, random_state=random_state)


In [28]:
X_train.dtypes

available_quantity                     int64
avg_gmv_item_domain_30days           float64
avg_gmv_item_sel                     float64
avg_gmv_seller_bday                  float64
avg_qty_orders_item_domain_30days    float64
avg_qty_orders_item_sel_30days       float64
avg_si_item_sel_30day                float64
health                               float64
offset                                 int64
original_price                         int64
price                                  int64
print_position                         int64
qty_items_dom                        float64
qty_items_sel                        float64
sold_quantity                          int64
total_asp_item_domain_30days         float64
total_asp_item_sel_30days            float64
total_gmv_domain_bday                float64
total_gmv_item_30days                float64
total_items_domain                     int64
total_items_seller                     int64
total_orders_domain_30days           float64
total_orde

In [30]:
#ver uan forma de tener mejor parametos que gerar
params = {'max_depth': list(range(3, 12)),
          'learning_rate': uniform(scale = 0.03),
          'gamma': uniform(scale=15),               #chiche va de 1 a 20
          'reg_lambda': uniform(scale = 15),        # Parámetro de regularización.
          'subsample': uniform(0.5, 0.5),          # Entre 0.5 y 1.
          'min_child_weight': uniform(scale = 50),   #0 a 100
          'colsample_bytree': uniform(0.65, 0.35), # Entre 0.75 y 1.
          'n_estimators': list(range(500, 2500))    #ma que 500 tambein a la noche para que corra
         }

diccionario_auc_grid = {'g': None, 'auc': None}
start = time.time()
best_score = 0
best_estimator = None
iterations = 20
for g in ParameterSampler(params, n_iter = iterations, random_state = random_state):
    clf_xgb = xgb.XGBClassifier(objective = 'binary:logistic', seed = random_state, eval_metric = 'auc', **g, tree_method = 'gpu_hist') #enable_categorical = True
    clf_xgb.fit(X_train, y_train, eval_set = [(X_val, y_val)], verbose = False)

    y_pred = clf_xgb.predict_proba(X_val)[:, 1] # Obtenemos la probabilidad de una de las clases (cualquiera).
    auc_roc = sklearn.metrics.roc_auc_score(y_val, y_pred)
    
    diccionario_auc_grid['auc'] = auc_roc
    diccionario_auc_grid['g'] = g
    print(f'Roc-AUC actual; {auc_roc}')
    print(f'Con grid: {g}')
    # Guardamos si es mejor.
    if auc_roc > best_score:
        print('-'*10)
        print(f'Mejor valor de ROC-AUC encontrado: {auc_roc}')
        print(f'Grid{g}')
        print('-'*10)
        best_score = auc_roc
        best_grid = g
        best_estimator = clf_xgb

end = time.time()



Roc-AUC actual; 0.8798928462394783
Con grid: {'colsample_bytree': 0.7810890415965769, 'gamma': 14.260714596148743, 'learning_rate': 0.02195981825434215, 'max_depth': 7, 'min_child_weight': 7.800932022121826, 'n_estimators': 966, 'reg_lambda': 1.4996237372700434, 'subsample': 0.7296244459829335}
----------
Mejor valor de ROC-AUC encontrado: 0.8798928462394783
Grid{'colsample_bytree': 0.7810890415965769, 'gamma': 14.260714596148743, 'learning_rate': 0.02195981825434215, 'max_depth': 7, 'min_child_weight': 7.800932022121826, 'n_estimators': 966, 'reg_lambda': 1.4996237372700434, 'subsample': 0.7296244459829335}
----------
Roc-AUC actual; 0.8908614195855594
Con grid: {'colsample_bytree': 0.7667980138986576, 'gamma': 2.143002268829112, 'learning_rate': 0.019526654188465584, 'max_depth': 7, 'min_child_weight': 48.49549260809972, 'n_estimators': 2015, 'reg_lambda': 14.078290635236252, 'subsample': 0.5003893829205072}
----------
Mejor valor de ROC-AUC encontrado: 0.8908614195855594
Grid{'colsa

In [31]:
print('ROC-AUC: %0.5f' % best_score)
print('Grilla:', best_grid)
print(f'Tiempo transcurrido: {str(end - start)} segundos')
print(f'Tiempo de entrenamiento por iteración: {str(round((end - start) / iterations, 2))} segundos')

ROC-AUC: 0.89149
Grilla: {'colsample_bytree': 0.7860370513913187, 'gamma': 4.070235476608438, 'learning_rate': 0.02486212527455788, 'max_depth': 11, 'min_child_weight': 14.046725484369038, 'n_estimators': 879, 'reg_lambda': 4.1399877303381505, 'subsample': 0.6481367528520412}
Tiempo transcurrido: 292.3969693183899 segundos
Tiempo de entrenamiento por iteración: 14.62 segundos


ahora con toda la data el prosible auc:

In [32]:
y_pred = best_estimator.predict_proba(X_val)[:, 1]
auc_roc = sklearn.metrics.roc_auc_score(y_val, y_pred)
print('AUC-ROC test: %0.5f' % auc_roc)

AUC-ROC test: 0.89149


entreno con toda la data train+val

In [33]:
clf_xgb = xgb.XGBClassifier(objective = 'binary:logistic', seed = random_state, eval_metric = 'auc', **best_grid,tree_method = 'gpu_hist') #enable_categorical = True
clf_xgb.fit(X_train_all, y_train_all, verbose = False)
final_model = clf_xgb

In [34]:

y_pred_submi = final_model.predict_proba(eval_data.drop(columns=["conversion", "ROW_ID"]))[:, 1]

Creo el submission

In [36]:
# y_preds = cls.predict_proba(eval_data.drop(columns=["ROW_ID"]))[:, cls.classes_ == 1].squeeze()
# Make the submission file
submission_df = pd.DataFrame({"ROW_ID": eval_data["ROW_ID"], "conversion": y_pred_submi})
submission_df["ROW_ID"] = submission_df["ROW_ID"].astype(int)
submission_df.to_csv("basicOHE_XGB_GS_29.csv", sep=",", index=False)